# 데이터 불러오기

In [1]:
import pandas as pd
import os
import re
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [2]:
def preprocess_df(df):
    df = df[['Numb', 'WAV', ' ', 'Segment ID', 'Total Evaluation', ' .1', ' .2']]
    df = df.rename(columns={'WAV':'WAV_start', ' ':'WAV_end', ' .1':'Valence', ' .2':'Arousal'})
    df.drop([0], inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df

In [5]:
path = './dataset/KEMDy20_v1_1/new/text'
sess_path = './dataset/KEMDy20_v1_1'

# annotation
ann = pd.read_csv(os.path.join(sess_path, 'annotation/Sess01_eval.csv'))
ann = preprocess_df(ann)
for i in range(2,41):
    idx_str = str(i)
    if i<10:
        idx_str = '0'+str(i)

    ann_session = pd.read_csv(os.path.join(sess_path, 'annotation/Sess'+idx_str+'_eval.csv'))
    ann_session = preprocess_df(ann_session)
    
    ann = pd.concat([ann, ann_session], axis=0)

ann.reset_index(inplace=True, )

In [6]:
ann

,index,Numb,WAV_start,WAV_end,Segment ID,Total Evaluation,Valence,Arousal
0,0,1,3.5783901,11.637391,Sess01_script01_User002M_001,neutral,3.4,2.9
1,1,2,11.637391,23.334393,Sess01_script01_User002M_002,neutral,3.1,2.9
2,2,3,23.334393,31.558392,Sess01_script01_User002M_003,neutral,3.1,3
3,3,4,31.558392,42.97039,Sess01_script01_User002M_004,neutral,3.7,3.1
4,4,5,42.77729,44.86329,Sess01_script01_User001F_001,neutral,3.8,2.8
...,...,...,...,...,...,...,...,...
13457,493,494,289.44998,294.943,Sess40_script06_User079F_043,neutral,3,3.5
13458,494,495,294.943,297.598,Sess40_script06_User079F_044,neutral,2.9,3.1
13459,495,496,297.598,304.171,Sess40_script06_User079F_045,neutral,2.6,3.6
13460,496,497,304.171,306.863,Sess40_script06_User079F_046,neutral,2.7,3.3


# 데이터 전처리

In [7]:
import re

text_list = []
len_list = []
for seg_id in tqdm(ann['Segment ID']):   
    sess = 'Session'+seg_id.split('_')[0][-2:]
    
    with open(os.path.join(sess_path, 'wav', sess, seg_id+'.txt'), 'r', encoding='CP949') as f:
        text = f.readlines()
        
        # 문장이 한 개 이상이면 멈춤
        if len(text) > 1:
            print('1문장 이상')
            break
        text = text[0]

        # 태그 삭제
        rep_words = ['\n','c/', 'n/', 'N/', 'u/', 'l/', 'b/', '*', '+', 'o/', 's/']
        for w in rep_words:
            text = text.replace(w, '')
        # 괄호 안 문자 삭제
        pattern = r'\([^)]*\)'
        text = re.sub(pattern, '', text)
        # 앞 뒤 공백 제거
        text = text.strip()
        # 공백 여러 개 생긴 경우 하나로 처리
        text = text.replace('  ', ' ')
        
        
        text_list.append(text)
        len_list.append(len(text))
        
ann['text'] = text_list
ann['len_text'] = len_list

100%|███████████████████████████████████████████████████████████████████████████| 13462/13462 [01:48<00:00, 124.00it/s]


In [8]:
print(min(ann['len_text']), max(ann['len_text']))
ann[['text', 'len_text']]

5 315


,text,len_text
0,"나는 어 어머니, 엄마가 이제 생일 때마다 이제 미역국도 많이 해서 미역국을 되게 ...",52
1,전에도 말했지만 그래서 미역국이랑 볶음김치 막 해주셔서 맨날 그렇게 차려주시는데 어...,86
2,어 응 거기를 되게 엄 집 앞에 그 조그만한 가게로 있었는데 거기를 엄청 생일 때마...,61
3,거기서 엄마랑 둘이 생일 파티 그냥 간단하게 하는 경우도 있었고 보 음 보통 그렇게...,98
4,아 친구들도?,7
...,...,...
13457,뭔가 막 그게 그니까 완전 대중화가 된 시대가 된 거야.,31
13458,그 게임 속 그 스토리 상에서는.,18
13459,"근데 그게 너무 저렴하고 하니까 다 갈아 치우고 막 내 아내처럼, 그니까 내가 아내...",54
13460,왜냐면 내 아내를 대신할 사람이 있는 거야.,24


In [14]:
text_df = ann[['Segment ID', 'text', 'len_text']].copy()
text_df.rename(columns={'Segment ID':'segment_id'}, inplace=True)
text_df

,segment_id,text,len_text
0,Sess01_script01_User002M_001,"나는 어 어머니, 엄마가 이제 생일 때마다 이제 미역국도 많이 해서 미역국을 되게 ...",52
1,Sess01_script01_User002M_002,전에도 말했지만 그래서 미역국이랑 볶음김치 막 해주셔서 맨날 그렇게 차려주시는데 어...,86
2,Sess01_script01_User002M_003,어 응 거기를 되게 엄 집 앞에 그 조그만한 가게로 있었는데 거기를 엄청 생일 때마...,61
3,Sess01_script01_User002M_004,거기서 엄마랑 둘이 생일 파티 그냥 간단하게 하는 경우도 있었고 보 음 보통 그렇게...,98
4,Sess01_script01_User001F_001,아 친구들도?,7
...,...,...,...
13457,Sess40_script06_User079F_043,뭔가 막 그게 그니까 완전 대중화가 된 시대가 된 거야.,31
13458,Sess40_script06_User079F_044,그 게임 속 그 스토리 상에서는.,18
13459,Sess40_script06_User079F_045,"근데 그게 너무 저렴하고 하니까 다 갈아 치우고 막 내 아내처럼, 그니까 내가 아내...",54
13460,Sess40_script06_User079F_046,왜냐면 내 아내를 대신할 사람이 있는 거야.,24


In [17]:
import pickle

pickle.dump(text_df, open(path+'/text_data.pkl', 'wb'))